## AmideSearch

In [1]:
pip install XlsxWriter # Added by EKG


  Using cached XlsxWriter-3.2.0-py3-none-any.whl.metadata (2.6 kB)
Using cached XlsxWriter-3.2.0-py3-none-any.whl (159 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import xlsxwriter # This line was added by EKG
import pandas as pd
import numpy as np
import sys

In [20]:
# Currently broken, only shifting column labels, not whole columns
def shifter(df, col_to_shift, pos_to_move):
    arr = df.columns.values
    idx = df.columns.get_loc(col_to_shift)
    if idx == pos_to_move:
        pass
    elif idx > pos_to_move:
        arr[pos_to_move+1: idx+1] = arr[pos_to_move: idx]
    else:
        arr[idx: pos_to_move] = arr[idx+1: pos_to_move+1]
    arr[pos_to_move] = col_to_shift
    df = df.reindex(columns=arr)
    return df

In [21]:
descriptionFileName = input("Enter name of the Description Table file: ")
if "Descriptions" in descriptionFileName:
    descriptionFile = open(descriptionFileName, "r")
else:
    print("The file you provided does not seem to be the Descriptions Table (CSV) file.")
    exit()
hitFileName = input("Enter name of the Hit Table file: ")
if "HitTable" in hitFileName:
    hitFile = open(hitFileName, "r")
else:
    print("The file you provided does not seem to be the Hit Table (CSV) file.")
    exit()

Enter name of the Description Table file:  B891P0X8016-Alignment-Descriptions.csv
Enter name of the Hit Table file:  B891P0X8016-Alignment-HitTable.csv


In [22]:
df_descriptionFile = pd.read_csv(descriptionFile)
df_hitFile = pd.read_csv(hitFile,header=None)

In [23]:
if len(df_hitFile)==len(df_descriptionFile):
    print("All good so far...")
else:
    print("Hmm, it looks like you don't have the same number of hits and descriptions.")

All good so far...


In [24]:
hitFileHeaders = ['Query code','Accession','% Identity','Align length','Mis-matches','Gap opens','Query start','Query end','Hit start','Hit end','E-value','Bit score','% positives']
df_labeledHitFile = df_hitFile.set_axis(hitFileHeaders,
                                        axis=1,
                                        copy=None)

In [25]:
df_noduplicateHitFile = df_labeledHitFile.drop_duplicates(subset='Accession',keep='first')
df_noduplicateHitFile.reset_index()

,index,Query code,Accession,% Identity,Align length,Mis-matches,Gap opens,Query start,Query end,Hit start,Hit end,E-value,Bit score,% positives
0,0,Query_3420593,XP_033127068.1,100.0,388,0,0,1,388,1,388,0.0,795,100.0


In [26]:
hitFileArray = pd.DataFrame.to_numpy(df_noduplicateHitFile)
matchedhitArray = []
for row in hitFileArray:
    if row[4] == 0:
        matchedhitArray.append(row)
dfMatchedHit = pd.DataFrame(matchedhitArray,columns = hitFileHeaders)
dfCleanedHit = dfMatchedHit.drop(hitFileHeaders[0], axis=1)
dfCleanedHit = dfCleanedHit.drop(hitFileHeaders[2:8], axis=1)

In [27]:
hitLengths = df_descriptionFile["Acc. Len"]
hitDescription = df_descriptionFile["Description"]
dfHitDescription = dfCleanedHit.join(hitLengths)
dfHitDescription = dfHitDescription.join(hitDescription)

In [28]:
# Currently broken, only shifting column labels, not whole columns
def shifter(df, col_to_shift, pos_to_move):
    arr = df.columns.values
    idx = df.columns.get_loc(col_to_shift)
    if idx == pos_to_move:
        pass
    elif idx > pos_to_move:
        arr[pos_to_move+1: idx+1] = arr[pos_to_move: idx]
    else:
        arr[idx: pos_to_move] = arr[idx+1: pos_to_move+1]
    arr[pos_to_move] = col_to_shift
    df = df.reindex(columns=arr)
    return df

In [29]:
sequenceFileName = input("Enter name of the sequence file: ")
if "seqdump" in sequenceFileName:
    sequenceFile = open(sequenceFileName, "r")
else:
    print("The file you provided does not seem to be the FASTA (completesequence) file.")
    exit()
sequences = sequenceFile.read()
sequences = sequences.split('\n')

Enter name of the sequence file:  seqdump.txt


In [30]:
sequenceLine = ''
for line in sequences:
    if '>' in line:
        sequenceLine = sequenceLine+','
    else:
        sequenceLine = sequenceLine+line

sequenceColumn = sequenceLine.split(',')[1:]
if len(sequenceColumn) == len(dfHitDescription):
    print('Good to go...')
elif len(sequenceColumn) > len(dfHitDescription):
    print('There are too many sequences.')
elif len(sequenceColumn) < len(dfHitDescription):
    print('There are not enough sequences.')
else:
    print('Something went wrong...')

Good to go...


In [31]:
dfHitDescription["Sequence"] = sequenceColumn

In [32]:
dfHitDescription

,Accession,Hit start,Hit end,E-value,Bit score,% positives,Acc. Len,Description,Sequence
0,XP_033127068.1,1,388,0.0,795,100.0,388,enterin neuropeptides-like [Anneissia japonica],MLRQPPFFLLLTWFLFQQCLLTLCYAGDNIHDAYNRPIGFPSKKAN...


In [33]:
outputSheetName = sequenceFileName.split('_')[0]
dfOutput = dfHitDescription.set_axis(range(1, len(dfHitDescription)+1))
with pd.ExcelWriter('AmideSearch.xlsx',
                    engine='xlsxwriter') as writer:  
    dfOutput.to_excel(writer, sheet_name=outputSheetName) # Changed forth line from mode='a' to engine='xlsxwriter'


In [34]:
dfOutput

,Accession,Hit start,Hit end,E-value,Bit score,% positives,Acc. Len,Description,Sequence
1,XP_033127068.1,1,388,0.0,795,100.0,388,enterin neuropeptides-like [Anneissia japonica],MLRQPPFFLLLTWFLFQQCLLTLCYAGDNIHDAYNRPIGFPSKKAN...


In [35]:
print(outputSheetName,"has been added to AmideSearch.xlsx")

seqdump.txt has been added to AmideSearch.xlsx
